In [76]:
import awkward as ak
import numpy as np
import uproot as up
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib.pyplot as plt
import correctionlib
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.lookup_tools import extractor
from coffea.analysis_tools import PackedSelection

fname = "/home/saikat/analysis/MC_files/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "ZGammaToLLGamma"},
).events()
totGenWeight = np.sum(events.genWeight)
totGenWeight

698608.0

# Working with $\mu$s

In [77]:
#Applying Trigger cuts 
hlt = events.HLT
#hlt.fields
trig_cut = np.stack((hlt.IsoMu24, hlt.IsoMu27), axis=1)
events = events[ak.any(trig_cut, axis=1)]
events

<NanoEventsArray [<event 1:132:91179>, ... ] type='204527 * event'>

In [78]:
#muon = events.Muon
#muon.fields
muon_sel = (abs(events.Muon.eta)<2.4) & (events.Muon.isGlobal==1) & (events.Muon.tightId==1) & (events.Muon.pfRelIso03_all<0.2) & (abs(events.Muon.dxy)<0.045) & (abs(events.Muon.dz)<0.2) & (ak.sum(events.Muon.charge, axis=1) == 0)
events.Muon = events.Muon[muon_sel]
#muon = muon[ak.num(muon)==2]
events = events[ak.num(events.Muon)==2]
events.Muon.pt

<Array [[134, 9.84], ... [47.7, 34.4]] type='97153 * var * float32[parameters={"...'>

In [79]:
#muon.fields

In [80]:
#Di-muon Invariant mass cut
mMuMu_cut = ((events.Muon[:,0]+events.Muon[:,1]).mass< 110) & ((events.Muon[:,0]+events.Muon[:,1]).mass> 70)
events = events[mMuMu_cut]
print(events.Muon)
print((events.Muon[:,0]+events.Muon[:,1]).mass)
print(mMuMu_cut)
events.Muon.pt

[[Muon, Muon], [Muon, Muon], [Muon, Muon, ... Muon], [Muon, Muon], [Muon, Muon]]
[92.5, 88.1, 89.8, 92.1, 72.1, 84.4, 80.5, ... 85.6, 90.7, 91.5, 91.5, 96, 91.3, 90]
[True, True, False, True, False, True, True, ... True, True, True, True, True, True]


<Array [[134, 9.84], ... [47.7, 34.4]] type='78792 * var * float32[parameters={"...'>

In [81]:
(events.Muon[:,0]+events.Muon[:,1]).mass
#events.Photon.pt

<Array [92.5, 88.1, 89.8, ... 96, 91.3, 90] type='78792 * float32'>

In [82]:
# Muon Gen Matching
genMatchMuon0 = events.Muon[:,0].delta_r(events.GenPart[(abs(events.GenPart.pdgId)==13) & ((events.GenPart.status==1) | (events.GenPart.status==23))])<0.4
genMatchMuon1 = events.Muon[:,1].delta_r(events.GenPart[(abs(events.GenPart.pdgId)==13) & ((events.GenPart.status==1) | (events.GenPart.status==23))])<0.4
isGenMatchMuons = (ak.any(genMatchMuon0, axis=1)) & (ak.any(genMatchMuon1, axis=1))
isGenMatchMuons
events = events[isGenMatchMuons]

# Working with $\gamma$

In [83]:
#Applying eta and other quality cuts
photon_sel = (events.Photon.pt>20) & (abs(events.Photon.eta)<2.5) & (events.Photon.pixelSeed==0) & (events.Photon.pfPhoIso03<0.2) & (events.Muon[:,0].delta_r(events.Photon)>0.4) & (events.Muon[:,1].delta_r(events.Photon)>0.4)
events.Photon = events.Photon[photon_sel]
#events = events[photon_sel]
events = events[ak.num(events.Photon)==1]
ak.num(events.Photon)
events.Photon.pt[ak.num(events.Photon)>1]

<Array [[31.6, 17.6, 10.1], ... [30.1, 10.8]] type='5333 * var * float32[paramet...'>

In [84]:
events = events[ak.num(events.Photon)<2]
ak.num(events.Photon)

<Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='6625 * int64'>

In [85]:
genMatchPhoton = events.Photon[:,0].delta_r(events.GenPart[(abs(events.GenPart.pdgId)==22) & ((events.GenPart.status==1) | (events.GenPart.status==23))])<0.1
ak.any(genMatchPhoton, axis=1)
events = events[ak.any(genMatchPhoton, axis=1)]

In [86]:
# Load the correctionlib JSON file #
evaluator = correctionlib.CorrectionSet.from_file("../EGmSFs/SS.json")
print(list(evaluator.keys()))
# Obtain the smearing values
evaluator_smearing = evaluator["Prompt2022FG_SmearingJSON"]
rho = evaluator_smearing.evaluate("rho", events.Photon[:,0].eta, events.Photon[:,0].r9)
rng = np.random.default_rng(seed=125)  # The smearing is done statistically, so we need some random numbers
smearing = rng.normal(loc=1., scale=rho)
photon_pt_smearing_nom = smearing * events.Photon[:,0].pt

['Prompt2022FG_ScaleJSON', 'Prompt2022FG_SmearingJSON']


In [87]:
# Getting the BDT distribution
import ROOT
import array
reader = ROOT.TMVA.Reader()
photon_esEffSigmaRR = array.array('f',[0]); reader.AddVariable("photon_esEffSigmaRR",photon_esEffSigmaRR)
photon_energyRaw = array.array('f',[0]); reader.AddVariable("photon_energyRaw",photon_energyRaw)
photon_esEnergyOverRawE = array.array('f',[0]); reader.AddVariable("photon_esEnergyOverRawE",photon_esEnergyOverRawE)
photon_etaWidth = array.array('f',[0]); reader.AddVariable("photon_etaWidth",photon_etaWidth)
photon_hoe = array.array('f',[0]); reader.AddVariable("photon_hoe",photon_hoe)
photon_phiWidth = array.array('f',[0]); reader.AddVariable("photon_phiWidth",photon_phiWidth)
photon_r9 = array.array('f',[0]); reader.AddVariable("photon_r9",photon_r9)
photon_s4 = array.array('f',[0]); reader.AddVariable("photon_s4",photon_s4)
photon_sieie = array.array('f',[0]); reader.AddVariable("photon_sieie",photon_sieie)
photon_sieip = array.array('f',[0]); reader.AddVariable("photon_sieip",photon_sieip)
photon_sipip = array.array('f',[0]); reader.AddVariable("photon_sipip",photon_sipip)
photon_rho = array.array('f',[0]); reader.AddVariable("photon_rho",photon_rho)

reader.BookMVA("BDT","/home/saikat/Softwares/root_v6.28.10.Linux-ubuntu22-x86_64-gcc11.4/root/tutorials/tmva/dataset_BDTG_weighted/weights/TMVAClassification_BDTG.weights.xml")

var_esEffSigmaRR = events.Photon.esEffSigmaRR[:,0]
var_energyRaw = events.Photon.energyRaw[:,0]
var_esEnergyOverRawE = events.Photon.esEnergyOverRawE[:,0]
var_etaWidth = events.Photon.etaWidth[:,0]
var_hoe = events.Photon.hoe[:,0]
var_phiWidth = events.Photon.phiWidth[:,0]
var_r9= events.Photon.r9[:,0]
var_s4= events.Photon.s4[:,0]
var_sieie= events.Photon.sieie[:,0]
var_sieip= events.Photon.sieip[:,0]
var_sipip= events.Photon.sipip[:,0]
var_rho = rho

bdtOutput = array.array('f')
for i in range(0,len(var_esEffSigmaRR)):
    photon_esEffSigmaRR[0] = var_esEffSigmaRR[i]
    photon_energyRaw[0] = var_energyRaw[i]
    photon_esEnergyOverRawE[0] = var_esEnergyOverRawE[i]
    photon_etaWidth[0] = var_etaWidth[i]
    photon_hoe[0] = var_hoe[i]
    photon_phiWidth[0] = var_phiWidth[i]
    photon_r9[0] = var_r9[i]
    photon_s4[0] = var_s4[i]
    photon_sieie[0] = var_sieie[i]
    photon_sieip[0] = var_sieip[i]
    photon_sipip[0] = var_sipip[i]
    photon_rho[0] = var_rho[i]
    
    bdtOutput.append(reader.EvaluateMVA("BDT"))

                         : Booking "BDT" of type "BDT" from /home/saikat/Softwares/root_v6.28.10.Linux-ubuntu22-x86_64-gcc11.4/root/tutorials/tmva/dataset_BDTG_weighted/weights/TMVAClassification_BDTG.weights.xml.
                         : Reading weight file: /home/saikat/Softwares/root_v6.28.10.Linux-ubuntu22-x86_64-gcc11.4/root/tutorials/tmva/dataset_BDTG_weighted/weights/TMVAClassification_BDTG.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTG" of type: "BDT"
                         : Rebuilding Dataset Default


In [88]:
file = up.recreate('output_ZGammaToMuMuGamma.root')
#file["tree3"] = {"Photon_pt": sel_photon_pt_smearing_nom,"nJet": nJet,"isJet":jet_pt_eta_cut,"Jet_pt": jet.pt}
file["tree"] = {"lead_muon": ak.zip({
                        "pt": events.Muon[:,0].pt,
                        "eta": events.Muon[:,0].eta,
                        "phi": events.Muon[:,0].phi,
                        "mass":events.Muon[:,0].mass,
                        }),
                "sublead_muon": ak.zip({
                        "pt": events.Muon[:,1].pt,
                        "eta": events.Muon[:,1].eta,
                        "phi": events.Muon[:,1].phi,
                        "mass":events.Muon[:,1].mass,
                        }),
                 
                 "dimuon_inv_mass":(events.Muon[:,0]+events.Muon[:,1]).mass,
                 "Photon_BDT": bdtOutput,
                 
                 "Photon": ak.zip({
                    "pt":events.Photon[:,0].pt,
                #     #"pt_smearing_up":sel_photon_pt_smearing_up,
                #    #"pt_smearing_down":sel_photon_pt_smearing_down,
                #    #"pt_scale_up":sel_photon_pt_scale_up,
                #    #"pt_scale_down":sel_photon_pt_scale_down,
                    "eta":events.Photon[:,0].eta,
                    "phi":events.Photon[:,0].phi,
                })
                }